# Part III Hand-on: Magnusson (2017) Paper

- Hands-on exercise for the StuMeTa 2021 Workshop "__A Practical Introduction to Ensemble Sensitivity Analysis__"
- Presentation by [Christopher Polster](mailto:cpolster@uni-mainz.de)
- Material: https://github.com/chpolste/ESA-Workshop


## Objective

Recreate plots from the paper "[Diagnostic methods for understanding the origin of forecast errors](https://doi.org/10.1002/qj.3072)" by Linus Magnusson (2017, QJRMS), specifically the ESA- and clustering-based results for case 3 from the paper (Fig. 13 a-c).

## Preparation

In [ ]:
import datetime as dt
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs

%matplotlib inline

In [ ]:
# Map projection
carree = crs.PlateCarree()

# Colormap setup for correlation-based sensitivity maps (based on Magnusson 2017)
corr_kwargs = {
    "levels": [-1., -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1.],
    "colors": ["#6D41A3", "#3758AF", "#57C9F5", "#FFFFFF", "#F9C719", "#F68026", "#EF282E"]
}

# Colormap setup for rank-based sensitivity maps (based on Magnusson 2017)
diff_kwargs = {
    "levels": [-3, -2.25, -1.5, -0.75, 0.75, 1.5, 2.25, 3],
    "colors": corr_kwargs["colors"]
}

# Contour setup for ensemble-mean 500 hPa Geopotential (based on Magnusson 2017)
z500_kwargs = {
    "levels": [4900, 5100, 5300, 5500, 5700, 5900],
    "colors": "black",
    "linewidths": 1
}

# add_axes with basic setup of a map
def add_map(fig, pos):
    ax = fig.add_axes(pos, projection=carree)
    ax.set_ylim((10, 90))
    ax.set_xlim((-180, 60))
    ax.coastlines()
    grid = ax.gridlines(
        xlocs=[-150, -120, -90, -60, -30, 0, 30],
        ylocs=[30, 60],
        draw_labels=True
    )
    grid.right_labels = False
    grid.top_labels = False
    return ax

Load the data:

In [ ]:
data = xr.load_dataset("data/data-2016-03-07.nc")
print(data)

The file contains two variables:

- `z500`: 500 hPa geopotential height (Z500) forecasts from the IFS ENS, initialized on 2016-03-07 00Z. Fields are available every 24 hours of lead time, out to 6 days (144 hours). These are the source fields in the sensitivity analysis.
- `rmse`: Forecast error of the 500 hPa geopotential height forecast in terms of the RMSE of every ensemble member relative to ERA5 over Europe (35°N-75°N, 12.5°W-42.5°E) evaluated at 144 hours lead time. This is our target metric in the sensitivity analysis. The larger this value, the worse the forecast for Europe.

Extract arrays that are needed for the analysis and plotting:

In [ ]:
# Coordinates for plotting
lat = data.latitude.values
lon = data.longitude.values

# The sensitivity target/forecast metric is the Z500 RMSE over Europe at +144 h
target = data.rmse.values

def z500(lead):
    """The Z500 fields for all members for a given lead time in hours"""
    valid = dt.datetime(2016, 3, 7, 0) + dt.timedelta(hours=lead)
    # Drop time, output dimensions are: number (member), latitude, longitude
    return data.z500.sel(time=valid, drop=True).values

---

## Sensitivity Maps

We follow Magnusson (2017) and assess sensitivities using correlation-based ESA.

---

**Task**: Implement correlation-based ESA. Compute the correlation between the source and target ensemble at every gridpoint:

$$ \mathrm{corr}(x_i, y) = \frac{\mathrm{cov}(x_i, y)}{\sigma_{x_i} \sigma_{y}} $$

Reminder: the (co)variances are evaluated along the ensemble dimension. Here, this is axis `0` of the source array (axis `1` is latitude and axis `2` is longitude). Note that [`np.cov`](https://numpy.org/doc/stable/reference/generated/numpy.cov.html) does not support the `axis` argument. you can use loops (the problem size considered here is small enough that this is feasible even with Python loops), or compute the covariance yourself with functions that have an `axis` argument.

In [ ]:
def esa_corr(source, target):
    ... # TODO

---

**Task**: Create a sensitivity map of Z500 vs. Z500 RMSE in Europe for +144h lead time.

In [ ]:
def correlation_map(lead):
    fig = plt.figure(figsize=(10, 5))
    ax = add_map(fig, (0.1, 0.2, 0.8, 0.8))
    cx = fig.add_axes((0.2, 0.2, 0.6, 0.03))
    
    source = ... # TODO
    mean   = ... # TODO
    corr   = ... # TODO
    
    # Correlations as filled contours
    cf = ax.contourf(lon, lat, corr, transform=carree, **corr_kwargs)
    plt.colorbar(cf, cax=cx, orientation="horizontal")
    
    # Ensemble Z500 mean as contours
    ct = ax.contour(lon, lat, mean, transform=carree, **z500_kwargs)
    ax.set_title("Sens. Corr +{} h".format(lead), loc="left")


correlation_map(144) # TODO

This is Fig. 13b of Magnusson (2017) without the significance test. Because we are evaluating both the target and source at the same time, it makes sense that we see the largest sensitivities in the region that was used to construct the target metric.

---


## Tracking Sensitivities

To follow the sensitivities of the forecast errors back through time, we decrease the lead time of the source Z500 fields, while keeping the valid time of the target RMSE values constant.

---

__Task__: Create sensitivity maps for earlier lead times out to +48h in 24h steps.

In [ ]:
... # TODO

The plot for +48h is Fig. 13a of Magnusson (2017).

In the paper it is shown that there is a cyclone developing at the location of the trough and that forecast errors related to the ridge over Europe decreased only when the development of the cyclone was finally captured correctly by the model. See the paper by [Grams et al. (2018)](http://doi.wiley.com/10.1002/qj.3353) for a more detailed investigation of this case.


## Comparison to Difference-based Sensitivites

Magnusson (2017) compares the ESA-derived Z500 sensitivity maps to ones obtained from a simple clustering technique: Take the 5 best performing members of the ensemble (best in terms of the RMSE) as well as the 5 worst-performing members. Compute the ensemble mean Z500 of each of these clusters and plot the (normalized) difference of these means.

---

**Task**: Implement rank-based ESA. Find the 5 members with the lowest and largest RMSE, compute the gridpoint-wise mean of each of corresponding source fields in these clusters and return the difference map, normalized by the gridpoint-wise standard deviation of the source fields.

$$ \mathrm{diff}(x_i, y) = \frac{\mathrm{mean}(x_{i,\mathrm{top}}) - \mathrm{mean}(x_{i,\mathrm{bot}})}{\sigma_{x_i}} $$

where the top and bot denote the 5 members with the largest and smalles values of $y$, respectively. The means and standard deviations are taken, as previously, along the ensemble dimension.

Hint: use [`np.argsort`](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) to obtain the indices of the high- and low-ranking ensemble members from the `target` array.

In [ ]:
def esa_diff(source, target):
    ... # TODO

---

__Task__: Plot sensitivities at +48h using the rank-based approach.

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = add_map(fig, (0.1, 0.2, 0.8, 0.8))
cx = fig.add_axes((0.2, 0.2, 0.6, 0.03))

diff = esa_diff(z500(48), target)
mean = z500(48).mean(axis=0)

# Difference-based sensitivities as filled contours
cf = ax.contourf(lon, lat, diff, transform=carree, **diff_kwargs) # TODO
plt.colorbar(cf, cax=cx, orientation="horizontal")

# Ensemble-mean Z500 as contours
ct = ax.contour(lon, lat, mean, transform=carree, **z500_kwargs) # TODO

ax.set_title("Sens. Rank +48 h", loc="left");

This is Fig. 13c of Magnusson (2017) minus the significance test.

---